In [1]:
import numpy as np

from scipy.stats import multivariate_normal

from sklearn.base import ClassifierMixin
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC

In [2]:
from time import time
from typing import List
from itertools import product

In [3]:
from utils import *

In [4]:
traces_path = "..\\acquisition\\50000_same_varying-mask_nullmask_maskshuffle\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Same-And-Varying-Multiple-Defenses.mat"
key_path = "..\\acquisition\\50000_same_varying-mask_nullmask_maskshuffle\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Same-And-Varying-Multiple-Defenses.log"

In [5]:
NUM_TRACES = 50_000
data_loader = EntireTraceIterator(traces_path, key_path, nr_populations=2, nr_scenarios=3, trace_size=850_000, traces_per_division=50_000)

In [6]:
EARLIEST_ROUND = 0

In [7]:
KEY_ALPHABET = list(range(16))

## Profiling stage

#### SOST

In [8]:
def sost_masks_card_m_v_y_subsample(seeds_sub: np.ndarray, key_shares_sub: np.ndarray, traces_sub: np.ndarray):
    assert seeds_sub.shape[0] == traces_sub.shape[0] == key_shares_sub.shape[0]
    assert key_shares_sub.shape[1:] == (KEY_WIDTH_B4, NR_SHARES)
    card_g = np.zeros((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET), 1), dtype=np.int32)
    m = np.zeros((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET), traces_sub.shape[1]), dtype=np.float32)
    v = np.zeros((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET), traces_sub.shape[1]), dtype=np.float64)
    y = np.zeros((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, NR_SHARES, traces_sub.shape[0]), dtype=int)

    for seed, key_shares, (trace_idx, trace) in zip(seeds_sub, key_shares_sub, enumerate(traces_sub)):
        indices, whitening = chacha_random_b4(seed)

        for round_idx in range(EARLIEST_ROUND, KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
            for block_idx in range(BLOCK_WIDTH_B4):
                keyround_index = round_idx * BLOCK_WIDTH_B4 + block_idx

                shares = np.zeros(NR_SHARES, dtype=int)
                for share_idx in range(shares.shape[0] - 1):
                    shares[share_idx] = key_shares[indices[keyround_index], share_idx]
                shares[-1] = (key_shares[indices[keyround_index], -1] + whitening[keyround_index]) % 16

                for share_idx, share in enumerate(shares):
                    card_g[round_idx, block_idx, share_idx, share, 0] += 1
                    m[round_idx, block_idx, share_idx, share] += trace
                    v[round_idx, block_idx, share_idx, share] += np.square(trace.astype(np.float64))

                    y[round_idx, block_idx, share_idx, trace_idx] = share
    
    return card_g, m, v, y

In [9]:
def sost_combine_subsamples(card_g: np.ndarray, m: np.ndarray, v: np.ndarray) -> np.ndarray:
    assert card_g.shape[3:] == (KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET), 1)
    assert m.ndim == 8 and m.shape[:7] == card_g.shape[:7]
    assert m.shape == v.shape
    f = np.zeros(m.shape[:-2] + (m.shape[-1],), dtype=np.float32)

    m /= card_g
    v = (v - card_g * m * m) / (card_g - 1)

    for i in range(len(KEY_ALPHABET)):
        for j in range(i + 1, len(KEY_ALPHABET)):
            num = m[..., i] - m[..., j]
            den = np.sqrt(v[..., i] / card_g[..., i] + v[..., j] / card_g[..., j])
            f += np.square(num / den)
    
    return f

In [10]:
def n_largest_values_separated_by_cycle(f: np.ndarray, n: int, cycle_length: int) -> np.ndarray:
    interesting_points_per_index = np.zeros((f.shape[0], f.shape[1], f.shape[2], n), dtype=int)
    sorted_strengths = np.argsort(f, axis=3)[:, :, :, ::-1]

    for round_idx in range(EARLIEST_ROUND, sorted_strengths.shape[0]):
        for block_idx in range(sorted_strengths.shape[1]):
            for share_idx in range(sorted_strengths.shape[2]):
                largest_indices = []
                for ind in sorted_strengths[round_idx, block_idx, share_idx]:
                    if len(largest_indices) < n:
                        if all(abs(i - ind) >= cycle_length for i in largest_indices):
                            largest_indices.append(ind)
                    else:
                        break
                interesting_points_per_index[round_idx, block_idx, share_idx] = np.array(largest_indices)
    
    return interesting_points_per_index

def n_largest_values(f: np.ndarray, n: int) -> np.ndarray:
    interesting_points_per_index = np.apply_along_axis(np.argpartition, axis=3, arr=f, kth=-n)[:, :, :, -n:]
    return interesting_points_per_index

In [ ]:
raise NotImplementedError
def means_and_covariance_large_dataset(data_loader: EntireTraceIterator, interesting_points_per_index: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    total_samples = np.zeros((len(KEY_ALPHABET),) + (interesting_points_per_index.shape[:3]), dtype=int)
    mean_vector = np.zeros(total_samples.shape + (interesting_points_per_index.shape[3],), dtype=np.float32)
    squared_dist_from_mean = np.zeros(total_samples.shape + (interesting_points_per_index.shape[3], interesting_points_per_index.shape[3]), dtype=np.float64)

    for seeds_sub, traces_sub, key, key_shares_sub in data_loader:
        traces_reduced = traces_sub[:, interesting_points_per_index].transpose(1, 2, 3, 0, 4)
        for k in len(KEY_ALPHABET):
            traces_reduced_per_key = traces_reduced[:, :, :, np.repeat((y == k)[:, :, :, :, np.newaxis], traces_reduced.shape[-1], axis=-1)]#.reshape(-1, traces_reduced.shape[-1])
                        
            # Welford's online algorithm
            total_samples[k] += traces_reduced_per_key.shape[-2]
            delta = np.subtract(traces_reduced_per_key, mean_vector[k])
            mean_vector[k] += np.sum(delta / total_samples[k])
            delta2 = np.subtract(traces_reduced_per_key, mean_vector[k])
            squared_dist_from_mean[k] = np.sum(delta * delta2)

    return np.moveaxis(mean_vector, 0, -1), np.moveaxis(squared_dist_from_mean / (total_samples - 1), 0, -1)

In [11]:
def recover_key_count_best(classifications_scores: np.ndarray, target_seeds: np.ndarray):
    classifications_best = np.argmax(classifications_scores, axis=-1)

    classifications_per_key_nibble = np.zeros((KEY_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET)), dtype=int)
    for i, seed in enumerate(target_seeds):
        indices, whitening = chacha_random_b4(seed)
        for keyround_index in range(KEYROUND_WIDTH_B4):
            key_index = indices[keyround_index]
            round_idx = keyround_index // BLOCK_WIDTH_B4
            block_idx = keyround_index % BLOCK_WIDTH_B4

            if round_idx >= EARLIEST_ROUND:
                for share_idx in range(NR_SHARES - 1):
                    classifications_per_key_nibble[key_index, share_idx, classifications_best[round_idx, block_idx, share_idx, i]] += 1
                classifications_per_key_nibble[key_index, NR_SHARES - 1, (classifications_best[round_idx, block_idx, NR_SHARES - 1, i] - whitening[keyround_index]) % 16] += 1

    recovered_key = np.argmax(classifications_per_key_nibble, axis=2) # Majority voting
    return np.sum(recovered_key, axis=1) % 16

def recover_key_sum_probs_rank(classifications_scores: np.ndarray, target_seeds: np.ndarray):
    classifications_ranks = np.apply_along_axis(lambda a: np.searchsorted(np.sort(-a), -a), axis=-1, arr=classifications_scores)

    classifications_per_key_nibble = np.zeros((KEY_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET)), dtype=int)
    for i, seed in enumerate(target_seeds):
        indices, whitening = chacha_random_b4(seed)
        for keyround_index in range(KEYROUND_WIDTH_B4):
            key_index = indices[keyround_index]
            round_idx = keyround_index // BLOCK_WIDTH_B4
            block_idx = keyround_index % BLOCK_WIDTH_B4

            if round_idx >= EARLIEST_ROUND:
                for share_idx in range(NR_SHARES - 1):
                    classifications_per_key_nibble[key_index, share_idx] += classifications_ranks[round_idx, block_idx, share_idx, i]
                classifications_per_key_nibble[key_index, NR_SHARES - 1] += np.roll(classifications_ranks[round_idx, block_idx, NR_SHARES - 1, i], -whitening[keyround_index])

    recovered_key = np.argmin(classifications_per_key_nibble, axis=2)
    return np.sum(recovered_key, axis=1) % 16

def recover_key_multiply_probs(classifications_scores: np.ndarray, target_seeds: np.ndarray):
    classifications_per_key_nibble = np.zeros((KEY_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET)), dtype=np.longdouble)
    for i, seed in enumerate(target_seeds):
        indices, whitening = chacha_random_b4(seed)
        for keyround_index in range(KEYROUND_WIDTH_B4):
            key_index = indices[keyround_index]
            round_idx = keyround_index // BLOCK_WIDTH_B4
            block_idx = keyround_index % BLOCK_WIDTH_B4

            if round_idx >= EARLIEST_ROUND:
                for share_idx in range(NR_SHARES - 1):
                    classifications_per_key_nibble[key_index, share_idx] += classifications_scores[round_idx, block_idx, share_idx, i]
                classifications_per_key_nibble[key_index, NR_SHARES - 1] += np.roll(classifications_scores[round_idx, block_idx, NR_SHARES - 1, i], -whitening[keyround_index])

    recovered_key = np.argmax(classifications_per_key_nibble, axis=2)
    return np.sum(recovered_key, axis=1) % 16

In [12]:
"""card_g = np.zeros((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET), 1), dtype=np.int32)
m = np.zeros((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET), data_loader.trace_size), dtype=np.float32)
v = np.zeros((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET), data_loader.trace_size), dtype=np.float64)
y = []

for seeds_sub, traces_sub, key, key_shares_sub in data_loader((1,), (0,)):
    card_g_sub, m_sub, v_sub, y_sub = sost_masks_card_m_v_y_subsample(seeds_sub[0][0], key_shares_sub[0][0], traces_sub[0][0])
    card_g += card_g_sub
    m += m_sub
    v += v_sub
    y.append(y_sub)

y = np.concatenate(y, axis=3)
f = sost_combine_subsamples(card_g, m, v)
with open("card_m_v_y_f_2_shares_masking.pic", "wb") as w:
    pic.dump((card_g, m, v, y, f), w)"""

In [ ]:
n_folds = 1
train_sizes = [45_000]
val_size_grid = [5_000]

grid = {"num_features": [40]}

In [ ]:
card_g = np.zeros((len(train_sizes), len(val_size_grid), n_folds, KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET), 1), dtype=np.int32)
m = np.zeros((len(train_sizes), len(val_size_grid), n_folds, KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET), data_loader.trace_size), dtype=np.float32)
v = np.zeros((len(train_sizes), len(val_size_grid), n_folds, KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, NR_SHARES, len(KEY_ALPHABET), data_loader.trace_size), dtype=np.float64)
y = [[[[] for cv in range(n_folds)] for m in range(len(val_size_grid))] for ts in range(len(train_sizes))]

for i, (seeds_sub, traces_sub, key, key_shares_sub) in enumerate(data_loader((1,), (0,))):
    for ts, train_size in enumerate(train_sizes):
        for m, val_size in enumerate(val_size_grid):
            rs = ShuffleSplit(n_splits=n_folds, test_size=val_size*data_loader.step//NUM_TRACES, train_size=train_size*data_loader.step//NUM_TRACES, random_state=i)
            for cv, (train_index, val_index) in enumerate(rs.split(traces_sub[0][0])):
                traces_train, seeds_train = traces_sub[0][0][train_index], seeds_sub[0][0][train_index]
                card_g_sub, m_sub, v_sub, y_sub = sost_masks_card_m_v_y_subsample(seeds_train, key_shares_sub[0][0], traces_train)
                card_g[ts, m, cv] += card_g_sub
                m[ts, m, cv] += m_sub
                v[ts, m, cv] += v_sub
                y[ts][m][cv].append(y_sub)

y = np.concatenate(y, axis=-1)
f = sost_combine_subsamples(card_g, m, v)
with open("card_m_v_y_f_2_shares_masking.pic", "wb") as w:
    pic.dump((card_g, m, v, y, f), w)

In [ ]:
def n_largest_values(f: np.ndarray, n: int) -> np.ndarray:
    interesting_points_per_index = np.apply_along_axis(np.argpartition, axis=2, arr=f, kth=-n)[:, :, -n:]
    return interesting_points_per_index


best_accuracy = -1
grid_results = np.zeros((len(grid["num_features"]), len(train_sizes), len(val_size_grid), f_trainval_tot.shape[1], n_folds), dtype=np.float64)
for l, num_features in enumerate(grid["num_features"]):
    for ts, train_size in enumerate(train_sizes):
        f_trainval_methods = f_trainval_tot[ts]
        y_trainval = y_trainval_tot[ts]

        for m, val_size in enumerate(val_size_grid):
            rs = ShuffleSplit(n_splits=n_folds, test_size=val_size, train_size=train_size, random_state=0)
            times = np.zeros((2, f_trainval_methods.shape[0], n_folds), dtype=np.float64)
            for cv, (train_index, val_index) in enumerate(rs.split(traces_trainval[ts])):
                print(f"CV {cv}", end="\r")
                traces_train, traces_val = traces_trainval[ts][train_index], traces_trainval[ts][val_index]
                seeds_train, seeds_val = seeds_trainval[ts][train_index], seeds_trainval[ts][val_index]
                y = y_trainval[m, cv]

                for n, f_trainval in enumerate(f_trainval_methods):
                    f = f_trainval[m, cv]

                    interesting_points_per_index = n_largest_values(f, num_features)
                    traces_reduced = traces_train[:, interesting_points_per_index].transpose(1, 2, 0, 3)
                    traces_val_reduced = traces_val[:, interesting_points_per_index].transpose(1, 2, 0, 3)

                    traces_reduced_per_key = [[[traces_reduced[round_idx, block_idx, np.repeat((y == k)[round_idx, block_idx, :, np.newaxis], traces_reduced.shape[-1], axis=-1)].reshape(-1, traces_reduced.shape[-1]) for k in range(len(KEY_ALPHABET))] for block_idx in range(traces_reduced.shape[1])] for round_idx in range(traces_reduced.shape[0])]

                    start = time()
                    template_means = np.array([[[np.mean(traces_reduced_per_key[round_idx][block_idx][k], axis=-2) for k in range(len(KEY_ALPHABET))] for block_idx in range(traces_reduced.shape[1])] for round_idx in range(traces_reduced.shape[0])])
                    template_covariances = np.array([[[np.nan_to_num(np.cov(traces_reduced_per_key[round_idx][block_idx][k], rowvar=False), nan=0.0) for k in range(template_means.shape[2])] for block_idx in range(template_means.shape[1])] for round_idx in range(template_means.shape[0])])
                    pdfs = np.array([[[multivariate_normal(template_means[round_idx, block_idx, k], template_covariances[round_idx, block_idx, k], allow_singular=True) for k in range(template_means.shape[2])] for block_idx in range(template_means.shape[1])] for round_idx in range(template_means.shape[0])])
                    times[0, n, cv] = time() - start
                    

                    start = time()
                    val_probas = np.array([[[[pdfs[round_idx, block_idx, key_guess].logpdf(traces_val_reduced[round_idx, block_idx, trace]) for key_guess in range(pdfs.shape[2])] for trace in range(traces_val_reduced.shape[2])] for block_idx in range(pdfs.shape[1])] for round_idx in range(pdfs.shape[0])])
                    recovered_key = recover_key_multiply_probs(val_probas, seeds_val)
                    times[1, n, cv] = time() - start

                    grid_results[l, ts, m, n, cv] = np.count_nonzero(recovered_key == real_keys[0]) / KEY_WIDTH_B4
                
            print(f"SOST [num features: {num_features}, train size: {train_size}, val size: {val_size}]: {np.mean(grid_results[l, ts, m, 0]):#.4g} ± {np.std(grid_results[l, ts, m, 0]):#.4g} ({grid_results[l, ts, m, 0]}). Training in {np.mean(times[0, 0]):#.4g} ± {np.std(times[0, 0]):#.4g} seconds. Extracting in {np.mean(times[1, 0]):#.4g} ± {np.std(times[1, 0]):#.4g} seconds.")
            if np.mean(grid_results[l, ts, m, 0]) > best_accuracy:
                print("New best model found ! (Above)")
                best_accuracy = np.mean(grid_results[l, ts, m, 0])


In [ ]:
with open("card_m_v_y_f_2_shares_masking.pic", "rb") as r:
    _, _, _, y_trainval_tot, f_trainval_tot = pic.load(r)

In [ ]:
grid_results = np.zeros((len(grid["num_features"]), len(train_sizes), len(val_size_grid), n_folds), dtype=np.float64)

best_accuracy = -1
for l, num_features in enumerate(grid["num_features"]):
    for ts, train_size in enumerate(train_sizes):
        f_trainval = f_trainval_tot[ts]
        y_trainval = y_trainval_tot[ts]
        for m, val_size in enumerate(val_size_grid):
            traces_train_reduced = np.zeros((n_folds, train_size, num_features), dtype=np.float32)
            traces_val_reduced = np.zeros((n_folds, val_size, num_features), dtype=np.float32)
            seeds_val = np.zeros((n_folds, val_size), dtype="<U32")
            for i, (seeds_sub, traces_sub, key, key_shares_sub) in enumerate(data_loader((1,), (0,))):
                real_key = key
                rs = ShuffleSplit(n_splits=n_folds, test_size=val_size*data_loader.step//NUM_TRACES, train_size=train_size*data_loader.step//NUM_TRACES, random_state=i)
                for cv, (train_index, val_index) in enumerate(rs.split(traces_sub[0][0])):
                    traces_train, traces_val = traces_sub[0][0][train_index], traces_sub[0][0][val_index]
                    y = y_trainval[m, cv]
                    f = f_trainval[m, cv]
                    interesting_points_per_index = n_largest_values(f, num_features)

                    traces_train_reduced[cv][i*data_loader.step//NUM_TRACES:(i+1)*data_loader.step//NUM_TRACES] = traces_train[:, interesting_points_per_index].transpose(1, 2, 3, 0, 4)
                    traces_val_reduced  [cv][i*data_loader.step//NUM_TRACES:(i+1)*data_loader.step//NUM_TRACES] = traces_val[:, interesting_points_per_index].transpose(1, 2, 3, 0, 4)
                    seeds_val[cv][i*data_loader.step//NUM_TRACES:(i+1)*data_loader.step//NUM_TRACES] = seeds_sub[0][0

            times = np.zeros((2, n_folds), dtype=np.float64)
            for cv in range(n_folds):
                start = time()
                for k in len(KEY_ALPHABET):
                    traces_reduced_per_key = traces_train_reduced[cv][..., np.repeat((y == k)[..., np.newaxis], traces_train_reduced.shape[-1], axis=-1)]#.reshape(-1, traces_reduced.shape[-1])

                    template_means = np.mean(traces_reduced_per_key[..., k], axis=-2)
                    template_covariances = np.array([[np.nan_to_num(np.cov(traces_reduced_per_key[round_idx][block_idx][k], rowvar=False), nan=0.0) for block_idx in range(template_means.shape[1])] for round_idx in range(template_means.shape[0])])
                    pdfs = np.array([[multivariate_normal(template_means[round_idx, block_idx, k], template_covariances[round_idx, block_idx, k], allow_singular=True) for block_idx in range(template_means.shape[1])] for round_idx in range(template_means.shape[0])])
                times[0, cv] = time() - start
                
                start = time()
                val_probas = np.array([[[[pdfs[round_idx, block_idx, key_guess].logpdf(traces_val_reduced[cv, round_idx, block_idx, trace]) for key_guess in range(pdfs.shape[2])] for trace in range(traces_val_reduced.shape[2])] for block_idx in range(pdfs.shape[1])] for round_idx in range(pdfs.shape[0])])
                recovered_key = recover_key_multiply_probs(val_probas, seeds_val[cv])
                times[1, cv] = time() - start

                grid_results[l, ts, m, cv] = np.count_nonzero(recovered_key == real_keys[0]) / KEY_WIDTH_B4
                
            print(f"SOST [num features: {num_features}, train size: {train_size}, val size: {val_size}]: {np.mean(grid_results[l, ts, m]):#.4g} ± {np.std(grid_results[l, ts, m]):#.4g} ({grid_results[l, ts, m]}). Training in {np.mean(times[0]):#.4g} ± {np.std(times[0]):#.4g} seconds. Extracting in {np.mean(times[1]):#.4g} ± {np.std(times[1]):#.4g} seconds.")
            if np.mean(grid_results[l, ts, m]) > best_accuracy:
                print("New best model found ! (Above)")
                best_accuracy = np.mean(grid_results[l, ts, m])
